In [5]:
#Imports:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from matplotlib import rcParams
from wordcloud import WordCloud

In [17]:
df_train = pd.read_csv('C:/Users/qw508/OneDrive/Documents/Covid-19 Twitter/client_labels_train.csv')
df_test = pd.read_csv('C:/Users/qw508/OneDrive/Documents/Covid-19 Twitter/client_labels_test.csv')
#combine train and test data into review_df
review_df = df_train.append(df_test, ignore_index=True)
#remove NA
review_df = review_df.dropna()
#convert irrelevant to 0 and relevant to 1
df_train.loc[(df_train['FINAL_LABEL'] == 'IRRELEVANT'), 'LABEL'] = 0
df_train.loc[(df_train['FINAL_LABEL'] != 'IRRELEVANT'), 'LABEL'] = 1  
df_test.loc[(df_test['FINAL_LABEL'] == 'IRRELEVANT'), 'LABEL'] = 0
df_test.loc[(df_test['FINAL_LABEL'] != 'IRRELEVANT'), 'LABEL'] = 1  
review_df.loc[(review_df['FINAL_LABEL'] == 'IRRELEVANT'), 'LABEL'] = 0
review_df.loc[(review_df['FINAL_LABEL'] != 'IRRELEVANT'), 'LABEL'] = 1  
print (review_df)

      X1            id FINAL_LABEL  \
0      1  1.251010e+18  IRRELEVANT   
1      2  1.250140e+18    RELEVANT   
2      3  1.250960e+18  IRRELEVANT   
4      5  1.248700e+18  IRRELEVANT   
5      6  1.248840e+18  IRRELEVANT   
6      7  1.249810e+18  IRRELEVANT   
7      8  1.248690e+18  IRRELEVANT   
8      9  1.249800e+18  IRRELEVANT   
9     10  1.248690e+18  IRRELEVANT   
10    11  1.249780e+18  IRRELEVANT   
11    13  1.250590e+18  IRRELEVANT   
12    14  1.249760e+18  IRRELEVANT   
13    16  1.250150e+18  IRRELEVANT   
14    17  1.250610e+18  IRRELEVANT   
15    18  1.249210e+18  IRRELEVANT   
16    20  1.249550e+18  IRRELEVANT   
17    21  1.250590e+18  IRRELEVANT   
18    23  1.250960e+18  IRRELEVANT   
19    24  1.250800e+18  IRRELEVANT   
20    25  1.250800e+18  IRRELEVANT   
22    27  1.249360e+18  IRRELEVANT   
23    28  1.250150e+18  IRRELEVANT   
24    29  1.249520e+18  IRRELEVANT   
25    30  1.249190e+18  IRRELEVANT   
26    32  1.250810e+18  IRRELEVANT   
27    33  1.

In [18]:
review_df['FINAL_LABEL'].value_counts()

IRRELEVANT    391
RELEVANT       78
Name: FINAL_LABEL, dtype: int64

In [19]:
#calculating baseline accuracy
review_df['FINAL_LABEL'].value_counts(normalize=True)

IRRELEVANT    0.833689
RELEVANT      0.166311
Name: FINAL_LABEL, dtype: float64

In [20]:
# Import Tokenizer
from nltk.tokenize import RegexpTokenizer

In [21]:
# Instantiate Tokenizer
tokenizer = RegexpTokenizer(r'\w+') 

In [22]:
#changing the contents of selftext to lowercase
review_df.loc[:,'text'] = review_df.text.apply(lambda x : str.lower(x))

In [23]:
#removing hyper link, latin characters and digits
review_df['text']=review_df['text'].str.replace('http.*.*', '',regex = True)
review_df['text']=review_df['text'].str.replace('û.*.*', '',regex = True)
review_df['text']=review_df['text'].str.replace(r'\d+','',regex= True)

In [24]:
review_df['tokens'] = review_df['text'].map(tokenizer.tokenize)

In [34]:
review_df.head()

,X1,id,FINAL_LABEL,text,LABEL,tokens
0,1,1.251010e+18,IRRELEVANT,i saw someone under that tweet say “no taxes n...,0.0,"[saw, someone, tweet, say, taxes, check, sir, ..."
1,2,1.250140e+18,RELEVANT,🧟‍♂️ \nscientists confirm first case of covid-...,1.0,"[scientists, confirm, first, case, covid, tran..."
2,3,1.250960e+18,IRRELEVANT,“the new requirements are bound to make face c...,0.0,"[new, requirements, bound, make, face, coverin..."
4,5,1.248700e+18,IRRELEVANT,🚨attention #smallbizdc community🚨\njoin dslbd ...,0.0,"[attention, smallbizdc, community, join, dslbd..."
5,6,1.248840e+18,IRRELEVANT,🥺 and i want my face in some pussy,0.0,"[want, face, pussy]"


In [26]:
# Printing English stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
#assigning stopwords to a variable
stop = stopwords.words("english")

In [28]:
# adding this stop word to list of stopwords as it appears on frequently occuring word
item=['amp'] #'https','co','http','û','ûò','ûó','û_'

In [29]:
stop.extend(item)

In [30]:
#removing stopwords from tokens
review_df['tokens']=review_df['tokens'].apply(lambda x: [item for item in x if item not in stop])

In [35]:
# Importing lemmatizer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Instantiating lemmatizer 
lemmatizer = WordNetLemmatizer()

In [50]:
review_df = review_df.reset_index(drop=True)
review_df.head()

,X1,id,FINAL_LABEL,text,LABEL,tokens
0,1,1.251010e+18,IRRELEVANT,i saw someone under that tweet say “no taxes n...,0.0,"[saw, someone, tweet, say, taxes, check, sir, ..."
1,2,1.250140e+18,RELEVANT,🧟‍♂️ \nscientists confirm first case of covid-...,1.0,"[scientists, confirm, first, case, covid, tran..."
2,3,1.250960e+18,IRRELEVANT,“the new requirements are bound to make face c...,0.0,"[new, requirements, bound, make, face, coverin..."
3,5,1.248700e+18,IRRELEVANT,🚨attention #smallbizdc community🚨\njoin dslbd ...,0.0,"[attention, smallbizdc, community, join, dslbd..."
4,6,1.248840e+18,IRRELEVANT,🥺 and i want my face in some pussy,0.0,"[want, face, pussy]"


In [38]:
review_df.head()

,X1,id,FINAL_LABEL,text,LABEL,tokens
0,1,1.251010e+18,IRRELEVANT,i saw someone under that tweet say “no taxes n...,0.0,"[saw, someone, tweet, say, taxes, check, sir, ..."
1,2,1.250140e+18,RELEVANT,🧟‍♂️ \nscientists confirm first case of covid-...,1.0,"[scientists, confirm, first, case, covid, tran..."
2,3,1.250960e+18,IRRELEVANT,“the new requirements are bound to make face c...,0.0,"[new, requirements, bound, make, face, coverin..."
4,5,1.248700e+18,IRRELEVANT,🚨attention #smallbizdc community🚨\njoin dslbd ...,0.0,"[attention, smallbizdc, community, join, dslbd..."
5,6,1.248840e+18,IRRELEVANT,🥺 and i want my face in some pussy,0.0,"[want, face, pussy]"


In [51]:
lemmatize_words=[]
for i in range (len(review_df['tokens'])):
    word=''
    for j in range(len(review_df['tokens'][i])):
        lemm_word=lemmatizer.lemmatize(review_df['tokens'][i][j])#lemmatize
        
        word=word + ' ' + lemm_word # joining tokens into sentence    
    lemmatize_words.append(word) # store in list

In [53]:
#creating a new column to store the result
review_df['lemmatized']=lemmatize_words

In [55]:
review_df.head()

,X1,id,FINAL_LABEL,text,LABEL,tokens,lemmatized
0,1,1.251010e+18,IRRELEVANT,i saw someone under that tweet say “no taxes n...,0.0,"[saw, someone, tweet, say, taxes, check, sir, ...",saw someone tweet say tax check sir pls choke...
1,2,1.250140e+18,RELEVANT,🧟‍♂️ \nscientists confirm first case of covid-...,1.0,"[scientists, confirm, first, case, covid, tran...",scientist confirm first case covid transmitte...
2,3,1.250960e+18,IRRELEVANT,“the new requirements are bound to make face c...,0.0,"[new, requirements, bound, make, face, coverin...",new requirement bound make face covering ines...
3,5,1.248700e+18,IRRELEVANT,🚨attention #smallbizdc community🚨\njoin dslbd ...,0.0,"[attention, smallbizdc, community, join, dslbd...",attention smallbizdc community join dslbd sma...
4,6,1.248840e+18,IRRELEVANT,🥺 and i want my face in some pussy,0.0,"[want, face, pussy]",want face pussy


In [56]:
#Constructing logistics Regression Model 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

In [57]:
X = review_df['lemmatized']
y = review_df['FINAL_LABEL']

In [58]:
# Spliting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [139]:
X_test.head()

418     roger stone bill gate may created coronavirus...
203     perversely greater lockdown succeeds stopping...
351     capitalweather positive note made social dist...
303      black make covid death dc mayor guess good mine
391     complacent covering severity crisis beginning...
Name: lemmatized, dtype: object

In [140]:
y_test.head()

418    IRRELEVANT
203    IRRELEVANT
351    IRRELEVANT
303    IRRELEVANT
391    IRRELEVANT
Name: FINAL_LABEL, dtype: object

In [59]:
#ensuring that the value counts are quite evenly distributed
y_train.value_counts()

IRRELEVANT    293
RELEVANT       58
Name: FINAL_LABEL, dtype: int64

In [60]:
y_test.shape

(118,)

In [61]:
pipe = Pipeline([
    ('cvec', CountVectorizer()),  
    ('lr', LogisticRegression()) 
])

In [62]:
tuned_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2,3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)]
}
gs = GridSearchCV(pipe, param_grid=tuned_params, cv=3) # Evaluating model on unseen data

model_lr=gs.fit(X_train, y_train) # Fitting model

# This is the average of all cv folds for a single 
#combination of the parameters specified in the tuned_params
print(gs.best_score_) 

#displaying the best values of parameters
gs.best_params_

C:\Users\qw508\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8319088319088319


{'cvec__max_df': 0.9,
 'cvec__max_features': 2500,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [63]:
# Test score
gs.score(X_train, y_train)

0.9515669515669516

In [64]:
# Test score
gs.score(X_test, y_test)

0.8135593220338984

In [65]:
# Generating predictions!
predictions_lr = model_lr.predict(X_test)

In [66]:
# Importing the confusion matrix function
from sklearn.metrics import confusion_matrix

In [67]:
# Generating confusion matrix
confusion_matrix(y_test, predictions_lr)

array([[95,  3],
       [19,  1]], dtype=int64)

In [68]:
#interpreting confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions_lr).ravel()

In [69]:
#values with coreesponding labels
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 95
False Positives: 3
False Negatives: 19
True Positives: 1


In [84]:
recall_logi = 1/(1+19)

In [91]:
accuracy_logi = (1+95)/(95+3+19+1)

In [92]:
recall_logi
accuracy_logi

0.8135593220338984

In [70]:
# Importing model- Navie Bayes Model
from sklearn.naive_bayes import MultinomialNB

In [71]:
# Instantiating model
nb = MultinomialNB()

In [72]:
# Instantiating CountVectorizer.
cvec = CountVectorizer(max_features = 500)

In [73]:
# fit_transform() fits the model and transforms training data into feature vectors
X_train_cvec = cvec.fit_transform(X_train, y_train).todense()

In [74]:
#tranform test data and convert into array
X_test_cvec = cvec.transform(X_test).todense()

In [75]:
# Fitting model
model_nb=nb.fit(X_train_cvec, y_train)

In [76]:
# Generating predictions
predictions_nb = model_nb.predict(X_test_cvec)

In [77]:
# Training score
model_nb.score(X_train_cvec, y_train)

0.9629629629629629

In [78]:
# Test score
model_nb.score(X_test_cvec, y_test)

0.8050847457627118

In [79]:
# Generating confusion matrix
confusion_matrix(y_test, predictions_nb)

array([[92,  6],
       [17,  3]], dtype=int64)

In [80]:
#interpreting confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions_nb).ravel()

In [81]:
#values with coreesponding labels
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 92
False Positives: 6
False Negatives: 17
True Positives: 3


In [88]:
recall_bayes = 3/(3+17)
accuracy_bayes = (3+92)/(92+6+17+3)

In [90]:
recall_bayes
accuracy_bayes

0.8050847457627118

In [94]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [98]:
%matplotlib inline

In [103]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)
%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))


Wall time: 0 ns
accuracy 0.8305084745762712


In [104]:
#Support Vector Machine
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

Wall time: 0 ns
accuracy 0.788135593220339


C:\Users\qw508\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [127]:
df_march = pd.read_csv('C:/Users/qw508/OneDrive/Documents/Covid-19 Twitter/march_tweet.csv')

In [128]:
df_march.shape

(1608, 6)

In [129]:
df_march.head()

,Unnamed: 0,id,Text,Date,Mentions,Hashtags
0,0,1244771412939702273,",uiiyk9, aPalo Alto church reports that a pers...",2020-03-30 23:40:06+00:00,NaN,NaN
1,1,1244767238961291264,I been breathing the Shoreview sewage air my w...,2020-03-30 23:23:31+00:00,NaN,NaN
2,2,1244766646700363777,@fredreker the next 2 months are gonna get sha...,2020-03-30 23:21:09+00:00,@fredreker,NaN
3,3,1244763563559759873,"Please, please stay at home. respect to those ...",2020-03-30 23:08:54+00:00,NaN,NaN
4,4,1244763456911200257,COVID-19 = OVER,2020-03-30 23:08:29+00:00,NaN,NaN


In [130]:
#remove NA
df_march = df_march.dropna()

In [131]:
#changing the contents of selftext to lowercase
df_march.loc[:,'Text'] = df_march.Text.apply(lambda x : str.lower(x))

In [132]:
#removing hyper link, latin characters and digits
df_march['Text']=df_march['Text'].str.replace('http.*.*', '',regex = True)
df_march['Text']=df_march['Text'].str.replace('û.*.*', '',regex = True)
df_march['Text']=df_march['Text'].str.replace(r'\d+','',regex= True)

In [133]:
df_march['tokens'] = df_march['Text'].map(tokenizer.tokenize)

In [134]:
#removing stopwords from tokens
df_march['tokens']=df_march['tokens'].apply(lambda x: [item for item in x if item not in stop])

In [135]:
df_march = df_march.reset_index(drop=True)
df_march.head()

,Unnamed: 0,id,Text,Date,Mentions,Hashtags,tokens
0,22,1244724436781031424,with the #boyking @realdonaldtrump as presiden...,2020-03-30 20:33:26+00:00,@realDonaldTrump,#BoyKing #COVID,"[boyking, realdonaldtrump, president, response..."
1,23,1244718490197487616,"thank you @gavinnewsom - for your thoughtful, ...",2020-03-30 20:09:48+00:00,@GavinNewsom,#FlattenTheCurve,"[thank, gavinnewsom, thoughtful, practical, de..."
2,29,1244699841243930630,worst perpetrators and schemers. cong n ak in ...,2020-03-30 18:55:42+00:00,@BloombergQuint,#ChineseVirus19,"[worst, perpetrators, schemers, cong, n, ak, c..."
3,36,1244663669750808578,thank you @startuphealth for having me on your...,2020-03-30 16:31:58+00:00,@startuphealth,#COVID19 #Covid_19 #healthcare,"[thank, startuphealth, covid, innovations, pod..."
4,59,1244469668275879936,my little contribution to the pm cares covid- ...,2020-03-30 03:41:04+00:00,@PMOIndia @narendramodi,#PMcaresfund #pmcares,"[little, contribution, pm, cares, covid, relie..."


In [136]:
lemmatize_words=[]
for i in range (len(df_march['tokens'])):
    word=''
    for j in range(len(df_march['tokens'][i])):
        lemm_word=lemmatizer.lemmatize(df_march['tokens'][i][j])#lemmatize
        
        word=word + ' ' + lemm_word # joining tokens into sentence    
    lemmatize_words.append(word) # store in list

In [137]:
#creating a new column to store the result
df_march['lemmatized']=lemmatize_words

In [138]:
df_march.head()

,Unnamed: 0,id,Text,Date,Mentions,Hashtags,tokens,lemmatized
0,22,1244724436781031424,with the #boyking @realdonaldtrump as presiden...,2020-03-30 20:33:26+00:00,@realDonaldTrump,#BoyKing #COVID,"[boyking, realdonaldtrump, president, response...",boyking realdonaldtrump president response co...
1,23,1244718490197487616,"thank you @gavinnewsom - for your thoughtful, ...",2020-03-30 20:09:48+00:00,@GavinNewsom,#FlattenTheCurve,"[thank, gavinnewsom, thoughtful, practical, de...",thank gavinnewsom thoughtful practical decisi...
2,29,1244699841243930630,worst perpetrators and schemers. cong n ak in ...,2020-03-30 18:55:42+00:00,@BloombergQuint,#ChineseVirus19,"[worst, perpetrators, schemers, cong, n, ak, c...",worst perpetrator schemer cong n ak cahoot sp...
3,36,1244663669750808578,thank you @startuphealth for having me on your...,2020-03-30 16:31:58+00:00,@startuphealth,#COVID19 #Covid_19 #healthcare,"[thank, startuphealth, covid, innovations, pod...",thank startuphealth covid innovation podcast ...
4,59,1244469668275879936,my little contribution to the pm cares covid- ...,2020-03-30 03:41:04+00:00,@PMOIndia @narendramodi,#PMcaresfund #pmcares,"[little, contribution, pm, cares, covid, relie...",little contribution pm care covid relief fund...


In [167]:
# Logistics Regression Model
X = df_march['lemmatized']

In [168]:
# Generating predictions from logistics Regression Model
predictions_march_lr = model_lr.predict(X)

In [172]:
df_march['Prediction'] = predictions_march_lr

In [173]:
df_march.loc[df_march.Prediction == 'RELEVANT'].count()

Unnamed: 0    8
id            8
Text          8
Date          8
Mentions      8
Hashtags      8
tokens        8
lemmatized    8
Prediction    8
dtype: int64

In [174]:
df_march.shape

(152, 9)

In [175]:
# about 5% relevant tweets
8/152 * 100 

5.263157894736842

In [141]:
# Naive Bays Model
X_test_cvec = df_march.lemmatized

In [144]:
#tranform march data and convert into array
X_test_cvec = cvec.transform(X_test_cvec).todense()

In [145]:
# Generating predictions from Naive Bays model 
predictions_march_nb = model_nb.predict(X_test_cvec)

In [152]:
df_march['Prediction'] = predictions_march_nb

In [162]:
df_march.head()

,Unnamed: 0,id,Text,Date,Mentions,Hashtags,tokens,lemmatized,Prediction
0,22,1244724436781031424,with the #boyking @realdonaldtrump as presiden...,2020-03-30 20:33:26+00:00,@realDonaldTrump,#BoyKing #COVID,"[boyking, realdonaldtrump, president, response...",boyking realdonaldtrump president response co...,IRRELEVANT
1,23,1244718490197487616,"thank you @gavinnewsom - for your thoughtful, ...",2020-03-30 20:09:48+00:00,@GavinNewsom,#FlattenTheCurve,"[thank, gavinnewsom, thoughtful, practical, de...",thank gavinnewsom thoughtful practical decisi...,IRRELEVANT
2,29,1244699841243930630,worst perpetrators and schemers. cong n ak in ...,2020-03-30 18:55:42+00:00,@BloombergQuint,#ChineseVirus19,"[worst, perpetrators, schemers, cong, n, ak, c...",worst perpetrator schemer cong n ak cahoot sp...,IRRELEVANT
3,36,1244663669750808578,thank you @startuphealth for having me on your...,2020-03-30 16:31:58+00:00,@startuphealth,#COVID19 #Covid_19 #healthcare,"[thank, startuphealth, covid, innovations, pod...",thank startuphealth covid innovation podcast ...,IRRELEVANT
4,59,1244469668275879936,my little contribution to the pm cares covid- ...,2020-03-30 03:41:04+00:00,@PMOIndia @narendramodi,#PMcaresfund #pmcares,"[little, contribution, pm, cares, covid, relie...",little contribution pm care covid relief fund...,IRRELEVANT


In [163]:
df_march.loc[df_march.Prediction == 'RELEVANT'].count()

Unnamed: 0    15
id            15
Text          15
Date          15
Mentions      15
Hashtags      15
tokens        15
lemmatized    15
Prediction    15
dtype: int64

In [164]:
df_march.shape

(152, 9)

In [165]:
#About 9% of Relevant tweets
15/152

0.09868421052631579

In [166]:
df_march.to_csv('C:/Users/qw508/OneDrive/Documents/march_tweet_labeled.csv')